In [1]:
import nltk
import pandas as pd
import numpy as np
import re
import tensorflow.keras as ke

In [2]:
!wget ivc0023.ivitco.com/makemytrip.com.zip

--2021-08-23 03:40:38--  http://ivc0023.ivitco.com/makemytrip.com.zip
Resolving ivc0023.ivitco.com (ivc0023.ivitco.com)... 192.99.8.125
Connecting to ivc0023.ivitco.com (ivc0023.ivitco.com)|192.99.8.125|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ivc0023.ivitco.com/makemytrip.com.zip [following]
--2021-08-23 03:40:39--  https://ivc0023.ivitco.com/makemytrip.com.zip
Connecting to ivc0023.ivitco.com (ivc0023.ivitco.com)|192.99.8.125|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18810098 (18M) [application/zip]
Saving to: ‘makemytrip.com.zip’

makemytrip.com.zip  100%[===================>]  17.94M  4.77MB/s    in 4.3s    

2021-08-23 03:40:44 (4.20 MB/s) - ‘makemytrip.com.zip’ saved [18810098/18810098]



In [3]:
!unzip makemytrip.com.zip

Archive:  makemytrip.com.zip
  inflating: promptcloud-hotels-on-makemytrip-com/datapackage.json  
  inflating: promptcloud-hotels-on-makemytrip-com/data/makemytrip_com_travel_sample.csv  
  inflating: promptcloud-hotels-on-makemytrip-com/original/makemytrip_com-travel_sample.csv  


In [4]:
data = pd.read_csv("/content/promptcloud-hotels-on-makemytrip-com/data/makemytrip_com_travel_sample.csv")

In [5]:
data.head()

,area,city,country,crawl_date,highlight_value,hotel_overview,hotel_star_rating,image_urls,in_your_room,is_value_plus,latitude,longitude,mmt_holidayiq_review_count,mmt_location_rating,mmt_review_count,mmt_review_rating,mmt_review_score,mmt_traveller_type_review_count,mmt_tripadvisor_count,pageurl,property_address,property_id,property_name,property_type,qts,query_time_stamp,room_types,site_review_count,site_review_rating,sitename,state,traveller_rating,uniq_id,location
0,Hardasji Ki Magri,Udaipur,India,2016-06-21,{{facility}},|Zion Home Stay is located in a city that sets...,1 star,NaN,{{value}},False,24.581788,73.667805,NaN,..,NaN,{{ratingCriteria.name}}{{ratingCriteria.value}},4.5,Families:{{ratingSummaryInfo.miscMap['family']...,6.0,http://hotelz.makemytrip.com/makemytrip/site/h...,"Zion, F-17, Haridas Ji Ki Magri, Near Bal Vina...",201302071713316867,Zion Home Stay,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,4.5,makemytrip,NaN,NaN,78ddf880bd7937d384ff278cc5b39d6e,POINT(73.667805 24.581788)
1,Near Nai Gaon,Udaipur,India,2016-06-21,{{facility}},| Araliayas Resorts is a 3 star hotel located ...,3 star,NaN,{{value}},False,24.544342,73.644242,NaN,..,NaN,{{ratingCriteria.name}}{{ratingCriteria.value}},4.5,Families:{{ratingSummaryInfo.miscMap['family']...,34.0,http://hotelz.makemytrip.com/makemytrip/site/h...,"Udaipur-Jhadol Road , Near Nai Gaon",201312041703404727,Araliayas Resorts,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,4.5,makemytrip,NaN,NaN,9f9f9cbb2f7df8089b63d5cdeb257944,POINT(73.644242 24.544342)
2,Near Bagore Ki Haveli,Udaipur,India,2016-06-21,{{facility}},|A 2 star property is located at 24 km from Ma...,2 star,NaN,{{value}},False,24.579942,73.681953,NaN,..,NaN,{{ratingCriteria.name}}{{ratingCriteria.value}},NaN,Families:{{ratingSummaryInfo.miscMap['family']...,16.0,http://hotelz.makemytrip.com/makemytrip/site/h...,"23/208 Gangaur Ghat Near Bagore Ki Haveli , Ne...",201407171324437682,Shri Udai Palace,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,NaN,makemytrip,NaN,NaN,b314bb7fa8bfb1ed306f517be21d729e,POINT(73.681953 24.579942)
3,Dabok,Udaipur,India,2016-06-21,Airport Transfer|Car rental|Conference Hall|Cu...,|SNP House Airport Hotel And Restaurant is loa...,1 star,//imghtlak.mmtcdn.com/images/hotels/2014071815...,NaN,False,24.622607,73.855194,0.0,NaN,NaN,NaN,NaN,Families:|Couples:|Business:|Solo:|Friends:,3.0,http://hotelz.makemytrip.com/makemytrip/site/h...,"Opp GITS, NH-76, Bamaniya Khet, Dabok , Dabok",201407181538181173,SNP House Airport Hotel And Restaurant,Hotel,NaN,2016-06-21 04:06:50 +0000,`standard,/5,NaN,makemytrip,NaN,NaN,e6f5bb3c2d76a78d978b9ceb0e31ec56,POINT(73.855194 24.622607)
4,East Udaipur,Udaipur,India,2016-06-21,{{facility}},| Hotel Pichola Haveli is situated in the beau...,2 star,NaN,{{value}},False,24.580100,73.682200,NaN,..,NaN,{{ratingCriteria.name}}{{ratingCriteria.value}},3.7,Families:{{ratingSummaryInfo.miscMap['family']...,139.0,http://hotelz.makemytrip.com/makemytrip/site/h...,"64 Ghangour Ghat, Opp Bagore Ki Haveli , East ...",201409021258573714,Hotel Pichola Haveli (LAKE SIDE),Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,3.7,makemytrip,NaN,NaN,63072c301427b6ca450d31eea127bcf0,POINT(73.6822 24.5801)


In [6]:
data = data.loc[data["city"].isin(["Mumbai"])]

In [7]:
data = data["hotel_overview"]

In [8]:
data = data.dropna()

In [9]:
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
def RemoverPalabrasSimples(fila):
  return [palabra for palabra in fila.split() if palabra not in nltk.corpus.stopwords.words("english")]

In [11]:
def LimpiarDatos(data):
  data_limpia = []
  palabras_unicas = []
  for doc in data:
    doc = re.sub("[^a-zA-Z]"," ",doc)
    doc = re.sub("\b\w{0,1}\b"," ",doc)
    doc = doc.lower()
    doc = RemoverPalabrasSimples(doc)
    data_limpia.append(doc)
    palabras_unicas.extend(list(set(doc)))
  return data_limpia,palabras_unicas

In [12]:
def PalabrasUnicas(todas_palabras_unicas):
  unicas = set(todas_palabras_unicas)
  return unicas, len(unicas)

In [13]:
data_limpia, palabras_unicas = LimpiarDatos(data)

In [14]:
palabras_unicas, n_palabras_unicas = PalabrasUnicas(palabras_unicas)

In [15]:
w2id = {}
id2w = {}
for i,palabra in enumerate(palabras_unicas):
  w2id[palabra] = i
  id2w[i] = palabra

In [16]:
w2id["good"]

3270

In [17]:
id2w[50]

'snooker'

In [18]:
def PrepararCorpus(corpus,w2id):
  secuencias = []
  for doc in corpus:
    for i in range(1, len(doc)):
      igram_seq = doc[:i+1]
      igram_seq_ids = []
      for j,token in enumerate(igram_seq):
        igram_seq_ids.append(w2id[token])
      secuencias.append(igram_seq_ids)
  return secuencias

In [19]:
secuencias = PrepararCorpus(data_limpia,w2id)

In [20]:
for i in range(10):
  print(secuencias[i])
  print([id2w[t] for t in secuencias[i] ])

[118, 1362]
['nestled', 'mumbai']
[118, 1362, 1830]
['nestled', 'mumbai', 'city']
[118, 1362, 1830, 557]
['nestled', 'mumbai', 'city', 'strong']
[118, 1362, 1830, 557, 2763]
['nestled', 'mumbai', 'city', 'strong', 'historical']
[118, 1362, 1830, 557, 2763, 2134]
['nestled', 'mumbai', 'city', 'strong', 'historical', 'links']
[118, 1362, 1830, 557, 2763, 2134, 2551]
['nestled', 'mumbai', 'city', 'strong', 'historical', 'links', 'wonderful']
[118, 1362, 1830, 557, 2763, 2134, 2551, 965]
['nestled', 'mumbai', 'city', 'strong', 'historical', 'links', 'wonderful', 'british']
[118, 1362, 1830, 557, 2763, 2134, 2551, 965, 2320]
['nestled', 'mumbai', 'city', 'strong', 'historical', 'links', 'wonderful', 'british', 'architecture']
[118, 1362, 1830, 557, 2763, 2134, 2551, 965, 2320, 2747]
['nestled', 'mumbai', 'city', 'strong', 'historical', 'links', 'wonderful', 'british', 'architecture', 'museums']
[118, 1362, 1830, 557, 2763, 2134, 2551, 965, 2320, 2747, 1512]
['nestled', 'mumbai', 'city', 'st

In [21]:
len(secuencias)

51925

In [22]:
len(data_limpia[0])

136

In [23]:
def ConstruirDataEntrada(secuencias, max_long_secuencia, n_palabras_unicas):
  secuencias = np.array( ke.preprocessing.sequence.pad_sequences(
      secuencias,max_long_secuencia) )
  x = secuencias[:,:-1]
  y = secuencias[:,-1]
  y = ke.utils.to_categorical(y,n_palabras_unicas)
  return x,y  

In [24]:
max_long_secuencias = max([len(s) for s in secuencias])

In [25]:
x,y = ConstruirDataEntrada(
    secuencias,
    max_long_secuencias,
    n_palabras_unicas
)

In [26]:
modelo = ke.models.Sequential()
modelo.add(ke.layers.Embedding(n_palabras_unicas, 10, input_length=max_long_secuencias-1 ))
modelo.add(ke.layers.LSTM(128))
modelo.add(ke.layers.Dropout(0.25))
modelo.add(ke.layers.Dense(n_palabras_unicas,activation="softmax"))
modelo.compile(loss="categorical_crossentropy", optimizer="adam")
modelo.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 307, 10)           34060     
_________________________________________________________________
lstm (LSTM)                  (None, 128)               71168     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 3406)              439374    
Total params: 544,602
Trainable params: 544,602
Non-trainable params: 0
_________________________________________________________________


In [ ]:
modelo.fit(x,y,batch_size=512,epochs=200)

Epoch 1/200
102/102 [==============================] - 21s 137ms/step - loss: 6.9355
Epoch 2/200
102/102 [==============================] - 14s 137ms/step - loss: 6.5693
Epoch 3/200
102/102 [==============================] - 14s 138ms/step - loss: 6.5011
Epoch 4/200
102/102 [==============================] - 14s 137ms/step - loss: 6.4433
Epoch 5/200
102/102 [==============================] - 14s 137ms/step - loss: 6.3722
Epoch 6/200
102/102 [==============================] - 14s 138ms/step - loss: 6.2913
Epoch 7/200
102/102 [==============================] - 14s 138ms/step - loss: 6.2140
Epoch 8/200
102/102 [==============================] - 14s 137ms/step - loss: 6.1398
Epoch 9/200
102/102 [==============================] - 14s 138ms/step - loss: 6.0187
Epoch 10/200
102/102 [==============================] - 14s 136ms/step - loss: 5.9019
Epoch 11/200
102/102 [==============================] - 14s 138ms/step - loss: 5.8886
Epoch 12/200
102/102 [==============================] - 14s 138

In [ ]:
def GenerarTexto(texto_inicial, n_palabras_para_generar, modelo,max_long_secuencias):
  c = texto_inicial
  for i in range(n_palabras_para_generar):
    data_limpia = LimpiarDatos([texto_inicial])
    secuencias = PrepararCorpus(data_limpia[0],w2id)
    secuencias = ke.preprocessing.sequence.pad_sequences([secuencias[-1]],maxlen=max_long_secuencias-1)
    prediccion = modelo.predict_classes(secuencias)
    salida = id2w[prediccion[0]]
    texto_inicial = texto_inicial + " " + salida
  return texto_inicial

In [ ]:
GenerarTexto("In Mumbai there is a good hotel",30,modelo,max_long_secuencias)